In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from proglearn import UncertaintyForest as UF
from hierarchical import HierarchicalForest

import time
from tqdm import tqdm
import pickle

from joblib import Parallel, delayed
import torchvision

from sklearn.model_selection import train_test_split


def get_accuracy(seed, fine_to_coarse, n_trees_coarse, n_trees_fine, max_depth):
    
    X_train, _, y_train, _ = train_test_split(trainset.data, trainset.targets, test_size=0.9, random_state=seed)
    
    if fine_to_coarse is None:
        f = UF(n_estimators=n_trees_fine, max_depth=max_depth).fit(X_train, y_train)
        return (f.predict(testset.data) == testset.targets).mean()
    
    f = HierarchicalForest(n_estimators_coarse=n_trees_coarse, n_estimators_fine=n_trees_fine, max_depth=max_depth)
    f.fit(X_train, y_train, fine_to_coarse)
    
    return (f.predict(testset.data) == testset.targets).mean()

In [2]:
#- Data processing 1
class Dataset:
    def __init__(self, file='cifar_100_Bit_m-r101x1_embd.p', train=True, classes=[]):
        if train:
            if file == '../../../data/cifar_100_Bit_m-r101x1_embd.p':
                self.data = pickle.load(open(file, 'rb'))[0][0]
                self.targets = np.concatenate(pickle.load(open(file, 'rb'))[0][1])
        else:
            if file == '../../../data/cifar_100_Bit_m-r101x1_embd.p':
                self.data = pickle.load(open(file, 'rb'))[1][0]
                self.targets = np.concatenate(pickle.load(open(file, 'rb'))[1][1])
        
        self.classes = classes

In [3]:
#- Data processing 2

cif100 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True)

file='../../../data/cifar_100_Bit_m-r101x1_embd.p'
trainset = Dataset(file, train=True, classes=cif100.classes)
testset = Dataset(file, train=False, classes=cif100.classes)

Files already downloaded and verified


In [4]:
#- Data processing 3

data_dimension=2048

if data_dimension < trainset.data.shape[1]:
    pca = PCA(n_components=data_dimension)
    pca.fit(trainset.data)
    trainset.data = pca.transform(trainset.data)
    testset.data = pca.transform(testset.data)

In [5]:
#- Data processing 4

coarse_to_fine_map = {
'aquatic_mammals': ['beaver', 'dolphin', 'otter', 'seal', 'whale'],
'fish': ['aquarium_fish', 'flatfish', 'ray', 'shark', 'trout'],
'flowers': ['orchid', 'poppy', 'rose', 'sunflower', 'tulip'],
'food_containers': ['bottle', 'bowl', 'can', 'cup', 'plate'],
'fruit_and_vegetables': ['apple', 'mushroom', 'orange', 'pear', 'sweet_pepper'],
'household_electrical_devices': ['clock', 'keyboard', 'lamp', 'telephone', 'television'],
'household_furniture': ['bed', 'chair', 'couch', 'table', 'wardrobe'],
'insects': ['bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach'],
'large_carnivores': ['bear', 'leopard', 'lion', 'tiger', 'wolf'],
'large_man-made_outdoor_things': ['bridge', 'castle', 'house', 'road', 'skyscraper'],
'large_natural_outdoor_scenes': ['cloud', 'forest', 'mountain', 'plain', 'sea'],
'large_omnivores_and_herbivores': ['camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo'],
'medium-sized_mammals': ['fox', 'porcupine', 'possum', 'raccoon', 'skunk'],
'non-insect_invertebrates': ['crab', 'lobster', 'snail', 'spider', 'worm'],
'people': ['baby', 'boy', 'girl', 'man', 'woman'],
'reptiles': ['crocodile', 'dinosaur', 'lizard', 'snake', 'turtle'],
'small mammals': ['hamster', 'mouse', 'rabbit', 'shrew', 'squirrel'],
'trees': ['maple_tree', 'oak_tree', 'palm_tree', 'pine_tree', 'willow_tree'],
'vehicles_1': ['bicycle', 'bus', 'motorcycle', 'pickup_truck', 'train'],
'vehicles_2': ['lawn_mower', 'rocket', 'streetcar', 'tank', 'tractor']
}

coarse_number_to_coarse_name = {i: name for i, name in enumerate(coarse_to_fine_map)}

def fine_to_coarse(coarse_to_fine):
    fine_to_coarse_map = {}
    for key in coarse_to_fine:
        fines = coarse_to_fine[key]
        for f in fines:
            fine_to_coarse_map[f] = key
            
    return fine_to_coarse_map

fine_to_coarse_map = fine_to_coarse(coarse_to_fine_map)

fine_number_to_fine_name = {i: name for i, name in enumerate(trainset.classes)}
fine_name_to_fine_number = {name: i for i, name in fine_number_to_fine_name.items()}

for i in range(100):
    fine_to_coarse_map[fine_number_to_fine_name[i]]
    
coarse_name_to_coarse_number = {name: i for i, name in enumerate(coarse_to_fine_map)}

coarse_targets = np.array([coarse_name_to_coarse_number[fine_to_coarse_map[fine_number_to_fine_name[y]]] for y in trainset.targets])
idx_by_coarse = np.array([np.where(coarse_targets == y)[0] for y in range(20)])
idx_by_fine = np.array([np.where(trainset.targets == y)[0] for y in range(100)])


test_coarse_targets = np.array([coarse_name_to_coarse_number[fine_to_coarse_map[fine_number_to_fine_name[y]]] for y in testset.targets])
test_idx_by_coarse = np.array([np.where(test_coarse_targets == y)[0] for y in range(20)])


coarse_names = np.array(list(coarse_name_to_coarse_number.keys()))

fine_number_to_coarse_number = {fn: coarse_name_to_coarse_number[
                                        fine_to_coarse_map[
                                            fine_number_to_fine_name[fn]
                                        ]
                                    ] for fn in range(100)}


fine_by_coarse = [np.where(np.array(list(fine_number_to_coarse_number.values())) == i)[0] for i in range(20)]
all_fine = np.concatenate(fine_by_coarse)

In [ ]:
n_mc=10
master_seed = 42
np.random.seed(master_seed)
seeds = np.random.randint(10000, size=n_mc)
n_cores=30

truth=np.array(list(fine_number_to_coarse_number.values()))

tasksim_clusters = pickle.load(open('cifar_tasksim_clusters.pkl', 'rb'))
condmean_clusters = pickle.load(open('cifar_condmean_clusters.pkl', 'rb'))

n_trees_coarse = 100
n_trees_fine = 25

max_depths=[10,20,30]

accuracy_tuples = []
for i, pair_of_clusters in enumerate(zip(tasksim_clusters, condmean_clusters)):
    n_classes = len(pair_of_clusters[0])
    
    n_tasksim_coarse = len(np.unique(pair_of_clusters[0]))
    n_condmean_coarse = len(np.unique(pair_of_clusters[0]))
    
    max_coarse = max([n_tasksim_coarse, n_condmean_coarse])

    for j, md in enumerate(max_depths):
        #- Hierarchical forest trained using tasksim clusters
        accuracy_tuples.append((seeds[i], pair_of_clusters[0], n_trees_coarse, n_trees_fine, md))

        #- Hierarchical forest trained using random clusters with the same structure as the tasksim clusters
        accuracy_tuples.append((seeds[i], 
                                pair_of_clusters[0][np.random.choice(n_classes, size=n_classes, replace=False)],
                              n_trees_coarse, n_trees_fine, md))


        #- Hierarchical forest trained using cond mean clusters
        accuracy_tuples.append((seeds[i], pair_of_clusters[1], n_trees_coarse, n_trees_fine, md))

        #- Hierarchical forest trained using random clusters with the same structure as the cond mean clusters
        accuracy_tuples.append((seeds[i], 
                                pair_of_clusters[1][np.random.choice(n_classes, size=n_classes, replace=False)],
                               n_trees_coarse, n_trees_fine, md))

        #- Flat forest
        accuracy_tuples.append((seeds[i], None, 0, n_trees_coarse + max_coarse * n_trees_fine, 2*md))
    
condensed_func = lambda x: get_accuracy(*x)
    
accuracies = Parallel(n_jobs=n_cores)(delayed(condensed_func)(tupl) for tupl in accuracy_tuples)

In [ ]:
tasksim_accuracies = np.zeros((len(max_depths), n_mc))
random_tasksim_accuracies = np.zeros((len(max_depths), n_mc))

condmean_accuracies = np.zeros((len(max_depths), n_mc))
random_condmean_accuracies = np.zeros((len(max_depths), n_mc))

flat_accuracies = np.zeros((len(max_depths), n_mc))

for i in range(int(len(accuracies) / 5)):
    index = i*5
    max_depth_idx = i % 3
    mc_idx = int(np.math.floor(i / 3))

    tasksim_accuracies[max_depth_idx, mc_idx] = accuracies[index+0]
    random_tasksim_accuracies[max_depth_idx, mc_idx] = accuracies[index+1]
    condmean_accuracies[max_depth_idx, mc_idx] = accuracies[index+2]
    random_condmean_accuracies[max_depth_idx, mc_idx] = accuracies[index+3]
    flat_accuracies[max_depth_idx, mc_idx] = accuracies[index+4]

print("(mean, std err) tasksim accuracy", np.mean(tasksim_accuracies), np.std(tasksim_accuracies) / np.sqrt(n_mc))
print("(mean, std err) random tasksim accuracy", np.mean(random_tasksim_accuracies), np.std(random_tasksim_accuracies) / np.sqrt(n_mc))
print("(mean, std err) condmean accuracy", np.mean(condmean_accuracies), np.std(condmean_accuracies) / np.sqrt(n_mc))
print("(mean, std err) random condmean accuracy", np.mean(random_condmean_accuracies), np.std(random_condmean_accuracies) / np.sqrt(n_mc))
print("(mean, std err) flat accuracy", np.mean(flat_accuracies), np.std(flat_accuracies) / np.sqrt(n_mc))